In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.feature_selection import RFE
from sklearn.metrics import recall_score, confusion_matrix, precision_score, f1_score, accuracy_score, classification_report

In [2]:
data = pd.read_csv('./churn_dataset.csv')

In [3]:
data = data.drop('customerID',axis=1)


In [4]:
display(data.head())
display(data.info())
display(data.describe())
display(data.shape)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 20 columns):
gender              7043 non-null object
SeniorCitizen       7043 non-null int64
Partner             7043 non-null object
Dependents          7043 non-null object
tenure              7043 non-null int64
PhoneService        7043 non-null object
MultipleLines       7043 non-null object
InternetService     7043 non-null object
OnlineSecurity      7043 non-null object
OnlineBackup        7043 non-null object
DeviceProtection    7043 non-null object
TechSupport         7043 non-null object
StreamingTV         7043 non-null object
StreamingMovies     7043 non-null object
Contract            7043 non-null object
PaperlessBilling    7043 non-null object
PaymentMethod       7043 non-null object
MonthlyCharges      7043 non-null float64
TotalCharges        7043 non-null object
Churn               7043 non-null object
dtypes: float64(1), int64(2), object(17)
memory usage: 1.1+ MB


None

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


(7043, 20)

The Data contains a mixture of Categorical and number

I could have used pandas dummies library to convert categorical data but it creates too many features so I didn't use that

In [5]:
def mapper_function(Y):
    x = data[Y].value_counts()
    title_mapping = {}
    counter = 0
    for i in x.keys():
        title_mapping[i] = counter
        counter += 1 
    data[Y] = data[Y].map(title_mapping)
    

In [6]:
mapper_function('Churn')
mapper_function('PaperlessBilling')
mapper_function('PhoneService')
mapper_function('MultipleLines')
mapper_function('InternetService')
mapper_function('OnlineSecurity')
mapper_function('Contract')
mapper_function('Partner')
mapper_function('Dependents')
mapper_function('gender')
mapper_function('TechSupport')
mapper_function('DeviceProtection')
mapper_function('PaymentMethod')
mapper_function('OnlineBackup')
mapper_function('StreamingTV')
mapper_function('StreamingMovies')

In [7]:
display(data.describe())


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,Churn
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.495244,0.162147,0.483033,0.299588,32.371149,0.096834,0.615505,0.777084,0.720006,0.778220,0.777226,0.723555,0.817691,0.821241,0.658952,0.407781,1.315633,64.761692,0.265370
std,0.500013,0.368612,0.499748,0.458110,24.559481,0.295752,0.656039,0.778877,0.796885,0.778472,0.778826,0.795896,0.763212,0.761725,0.801943,0.491457,1.148907,30.090047,0.441561
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,18.250000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,9.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.500000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,29.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,70.350000,0.000000
75%,1.000000,0.000000,1.000000,1.000000,55.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,89.850000,1.000000
max,1.000000,1.000000,1.000000,1.000000,72.000000,1.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,1.000000,3.000000,118.750000,1.000000


In [8]:
data['TotalCharges'] = data['TotalCharges'].apply(pd.to_numeric, errors='coerce') 

Feature Correlation Matrix

In [9]:
data.corr().style.background_gradient(cmap='PuBu', low=0, high=0, axis=0, subset=None)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
gender,1,0.00187371,0.00180808,-0.0105166,-0.00510622,-0.00648767,0.000484598,-0.0103802,0.00342855,0.00203218,-0.00509166,-0.000985182,-0.00115629,0.000190735,-0.00617098,-0.0117537,0.00520929,0.0145693,-4.78395e-05,0.0086121
SeniorCitizen,0.00187371,1,0.0164787,-0.211185,0.0165669,-0.0085764,0.0998834,-0.25939,-0.210897,-0.15278,-0.157095,-0.22377,-0.13013,-0.120802,-0.109299,-0.15653,-0.093704,0.220173,0.102411,0.150889
Partner,0.00180808,0.0164787,1,0.452676,0.379697,-0.0177057,0.0909813,0.000131882,0.0818497,0.087055,0.0944515,0.0690715,0.080127,0.0757787,0.216229,0.0148766,0.133115,0.0968479,0.319072,-0.150448
Dependents,-0.0105166,-0.211185,0.452676,1,0.159712,0.00176168,-0.0168755,0.179631,0.190523,0.162445,0.156439,0.180832,0.140395,0.12582,0.178429,0.111377,0.123844,-0.11389,0.0646532,-0.164221
tenure,-0.00510622,0.0165669,0.379697,0.159712,1,-0.00844821,0.242279,-0.0332304,0.145298,0.178651,0.178649,0.144459,0.136145,0.140781,0.503232,-0.00615248,0.340305,0.2479,0.82588,-0.352229
PhoneService,-0.00648767,-0.0085764,-0.0177057,0.00176168,-0.00844821,1,0.69107,0.0937203,-0.125353,-0.150338,-0.138755,-0.12335,-0.171538,-0.165205,0.000954757,0.0165048,0.00406994,-0.247398,-0.113008,-0.011942
MultipleLines,0.000484598,0.0998834,0.0909813,-0.0168755,0.242279,0.69107,1,-0.174989,-0.235021,-0.210372,-0.200463,-0.232155,-0.202414,-0.195815,0.040607,-0.10823,0.0306595,0.146153,0.251357,0.019423
InternetService,-0.0103802,-0.25939,0.000131882,0.179631,-0.0332304,0.0937203,-0.174989,1,0.764943,0.686403,0.682448,0.760985,0.617704,0.620005,0.203148,0.378108,0.178137,-0.905491,-0.428212,-0.316846
OnlineSecurity,0.00342855,-0.210897,0.0818497,0.190523,0.145298,-0.125353,-0.235021,0.764943,1,0.751661,0.74904,0.791225,0.701976,0.704984,0.275864,0.334003,0.2138,-0.621227,-0.153234,-0.332819
OnlineBackup,0.00203218,-0.15278,0.087055,0.162445,0.178651,-0.150338,-0.210372,0.686403,0.751661,1,0.740604,0.754095,0.720671,0.7167,0.252097,0.262402,0.182909,-0.538454,-0.0850173,-0.291449


There is lot of high correlated features but I'm not going to drop, will be using feature selection method to handle it

Checking if there is any null value in the table.

In [10]:
data.isnull().sum()

gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

There is 11 Missing data in the dataset in Total Charges:
which is around 0.15 % of the data
There is a high correlation between tenure and Totalcharges, so will groupby tenure and take mean

In [11]:
# data["TotalCharges"].fillna(data.groupby("tenure")["TotalCharges"].transform("mean"), inplace=True)
df = data

In [12]:
df["TotalCharges"] = df.groupby("tenure").transform(lambda x: x.fillna(x.mean()))

Feature Normalization

In [13]:
y = df['Churn']
x = df.iloc[:, :-1]
x = (x-np.min(x))/(np.max(x)-np.min(x)).values
x.shape

(7043, 19)

Feature selection based on Principal Components Analysis

In [14]:
pca = PCA(0.95)  
principalComponents = pca.fit_transform(x) 
pca.explained_variance_ratio_.cumsum()
principalDf = pd.DataFrame(data = principalComponents)

In [15]:
x = principalDf 
x.shape

(7043, 12)

In [16]:
x = (x-np.min(x))/(np.max(x)-np.min(x)).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state =1)


In [17]:
model = LogisticRegression(max_iter=100,solver='lbfgs')
model.fit(x_train,y_train)
accu = model.score(x_test,y_test)
print("Logistic Regression accuracy is :" + str(accu))


print("f1 Score for Logistic Regression :",f1_score(y_test, model.predict(x_test), average='weighted') )

Logistic Regression accuracy is :0.8097941802696949
f1 Score for Logistic Regression : 0.8072547721286278


In [18]:
report = classification_report(y_test, model.predict(x_test))
print(report)

              precision    recall  f1-score   support

           0       0.86      0.89      0.88      1061
           1       0.62      0.58      0.60       348

   micro avg       0.81      0.81      0.81      1409
   macro avg       0.74      0.73      0.74      1409
weighted avg       0.81      0.81      0.81      1409





Feature selection based on Recursive Feature Elimination (RFE), I choose RFE since the number of features and dataset is small it shouldn't make much time to compute and it is better than Forward and Back feature selection in most cases 



In [19]:
y = df['Churn']
x = df.iloc[:, :-1]
x = (x-np.min(x))/(np.max(x)-np.min(x)).values
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2,random_state =1)

In [20]:
lm = LogisticRegression(max_iter=100,solver='lbfgs')
# score_
accuracy = []
f1score = []
final_set_features = []
for i in range(2,20):
    rfe = RFE(lm, i)
    rfe = rfe.fit(x_train, y_train)
    final_list = []
    final_vars= x_train.columns.values.tolist()
    j=0
    for i in final_vars:
        if rfe.support_[j] == True :
            final_list.append(i)    
        j = j + 1
    fx_train = x_train[final_list]
    fx_test = x_test[final_list]
    lm.fit(fx_train, y_train)
    final_set_features.append(final_list)
    pred = lm.predict(fx_test)
    scores = lm.score(fx_test,y_test)
    accuracy.append(scores)
    f1score.append(f1_score(y_test, pred, average='weighted'))


In [21]:
print("Result based on the highest F1 score")
index = f1score.index(max(f1score))
print("Logistic Regression score is.:", accuracy[index] )
print("F1score for the best feature selected", (f1score[index]) )

Result based on the highest F1 score
Logistic Regression score is.: 0.8090844570617459
F1score for the best feature selected 0.8064312311041753


Selected Features based based on RFE:

In [22]:
print(final_set_features[index])

['SeniorCitizen', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges']
